<a href="https://colab.research.google.com/github/akshay-3apr/UoG-Dissertation/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Pyterrier

In [4]:
%pip install -q python-terrier git+https://github.com/cmacdonald/pyterrier_colbert.git faiss-gpu==1.6.3

     |████████████████████████████████| 97 kB 3.1 MB/s 
     |████████████████████████████████| 35.5 MB 419 kB/s 
     |████████████████████████████████| 769 kB 49.9 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 17.9 MB 649 kB/s 
     |████████████████████████████████| 1.2 MB 38.0 MB/s 
     |████████████████████████████████| 3.0 MB 37.9 MB/s 
     |████████████████████████████████| 880 kB 58.6 MB/s 
     |████████████████████████████████| 1.1 MB 12.3 MB/s 
     |████████████████████████████████| 69 kB 6.6 MB/s 
     |████████████████████████████████| 300 kB 49.7 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 126 kB 45.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |██████████

## Import pyterrier and initialization

In [5]:
import pyterrier as pt
import pandas as pd
import sys

COLAB='google.colab' in sys.modules

try:
  import faiss
  faiss.get_num_gpus()
except:
  if COLAB:
    print('Installing faiss-gpu from pip ')
    %pip install faiss-gpu==1.6.3
  else:
    print('Installing faiss-gpu via Conda')
    %conda install -c pytorch faiss-gpu

# checks if pyterrier init method is called
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



### Mount google drive

In [6]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


### Datasets

In [ ]:
## Get list of dataset
pt.datasets.find_datasets('msmarco')

[INFO] trec-robust04 is deprecated. Consider using disks45/nocr/trec-robust-2004 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold1 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold1 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold2 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold2 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold3 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold3 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold4 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold4 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold5 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold5 instead, which provides better parsing of the corpus.
[INFO] acessing TREC Fair Ranking 2021 through trec-fair-2021 is deprecated; use trec-fair/2021 instead.
[INFO] acessing TREC Fair

,dataset,topics,topics_lang,qrels,corpus,corpus_lang,index,info_url
3,msmarco_document,"[train, dev, test, test-2020, leaderboard-2020]",en,"[train, dev, test, test-2020]",True,en,True,https://microsoft.github.io/msmarco/
4,msmarcov2_document,"[train, dev1, dev2, valid1, valid2, trec_2021]",en,"[train, dev1, dev2, valid1, valid2]",None,None,True,https://microsoft.github.io/msmarco/TREC-Deep-...
5,msmarco_passage,"[train, dev, dev.small, eval, eval.small, test...",en,"[train, dev, test-2019, test-2020, dev.small]",True,en,True,https://microsoft.github.io/MSMARCO-Passage-Ra...
6,msmarcov2_passage,"[train, dev1, dev2, trec_2021]",en,"[train, dev1, dev2]",None,None,True,https://microsoft.github.io/msmarco/TREC-Deep-...
69,irds:beir/msmarco,True,en,None,True,en,None,https://ir-datasets.com/beir.html#beir/msmarco
...,...,...,...,...,...,...,...,...
446,irds:msmarco-passage-v2/trec-dl-2021/judged,True,en,True,True,en,None,https://ir-datasets.com/msmarco-passage-v2.htm...
447,irds:msmarco-qna,None,None,None,True,en,None,https://ir-datasets.com/msmarco-qna.html
448,irds:msmarco-qna/dev,"[text, type, answers]",en,True,True,en,None,https://ir-datasets.com/msmarco-qna.html#msmar...
449,irds:msmarco-qna/eval,"[text, type]",en,None,True,en,None,https://ir-datasets.com/msmarco-qna.html#msmar...


####Importing data

In [54]:
## Immporting data

dataset = pt.get_dataset("msmarco_passage")
indexref = dataset.get_index(variant='terrier_stemmed')
index = pt.IndexFactory.of(indexref)

# tr_topics = dataset.get_topics("train") 
# va_topics = dataset.get_topics("dev.small")

# tr_qrels = dataset.get_qrels("train")
# va_qrels = dataset.get_qrels("dev.small")

test_topics_19 = dataset.get_topics("test-2019")
test_qrels_19 = dataset.get_qrels("test-2019")

test_topics_20 = dataset.get_topics("test-2020")
test_qrels_20 = dataset.get_qrels("test-2020")

print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [ ]:
print("Count of unique queries: ",test_qrels_19.qid.unique().size)
test_qrels_19.merge(test_topics_19, left_on='qid', right_on='qid')

Count of unique queries:  43


,qid,docno,label,query
0,19335,1017759,0,anthropological definition of environment
1,19335,1082489,0,anthropological definition of environment
2,19335,109063,0,anthropological definition of environment
3,19335,1160863,0,anthropological definition of environment
4,19335,1160871,0,anthropological definition of environment
...,...,...,...,...
9255,1133167,8839920,2,how is the weather in jamaica
9256,1133167,8839922,2,how is the weather in jamaica
9257,1133167,944810,0,how is the weather in jamaica
9258,1133167,949411,0,how is the weather in jamaica


In [ ]:
print("Count of unique queries: ",test_qrels_20.qid.unique().size)
test_qrels_20.merge(test_topics_20, left_on='qid', right_on='qid')

Count of unique queries:  54


,qid,docno,label,query
0,23849,1020327,2,are naturalization records public information
1,23849,1034183,3,are naturalization records public information
2,23849,1120730,0,are naturalization records public information
3,23849,1139571,1,are naturalization records public information
4,23849,1143724,0,are naturalization records public information
...,...,...,...,...
11381,1136962,8526087,0,why did the ancient egyptians call their land ...
11382,1136962,8537921,0,why did the ancient egyptians call their land ...
11383,1136962,8742482,0,why did the ancient egyptians call their land ...
11384,1136962,937258,1,why did the ancient egyptians call their land ...


####Export MSMARCO pyterrier datatset

In [98]:
test_qrels_19['q']="0"
test_qrels_19[['qid','q','docno', 'label']].to_csv("test_qrels.tsv",sep="\t",header=False,index=False)
!cp -av 'test_qrels.tsv' '/content/mydrive/MyDrive/Colab/Dissertation'
test_topics_19[test_topics_19.qid.isin(test_qrels_19.qid)].to_csv("test_topics.tsv",sep="\t",header=False,index=False)
!cp -av 'test_topics.tsv' '/content/mydrive/MyDrive/Colab/Dissertation'

'test_qrels.tsv' -> '/content/mydrive/MyDrive/Colab/Dissertation/test_qrels.tsv'
'test_topics.tsv' -> '/content/mydrive/MyDrive/Colab/Dissertation/test_topics.tsv'


In [99]:
##importing test_qrels and topics from google drive

test_topics = pd.read_csv('/content/mydrive/MyDrive/Colab/Dissertation/test_topics.tsv',sep='\t',header=None,names=['qid','query'])
test_qrels = pd.read_csv('/content/mydrive/MyDrive/Colab/Dissertation/test_qrels.tsv',sep='\t',header=None,names=['qid','q','docno', 'label'])

#### Qrels labels relevance:

* [3] Perfectly relevant: The passage is dedicated to the query and contains the exact answer.
* [2] Highly relevant: The passage has some answer for the query, but the answer may be a bit unclear, or hidden
amongst extraneous information.
* [1] Related: The passage seems related to the query but does not answer it.
* [0] Irrelevant: The passage has nothing to do with the query.





##BM25 Model

In [ ]:
## BM25 retrieval model
bm25 = pt.BatchRetrieve(index,num_results=1000, wmodel="BM25")

In [ ]:
retrieved_doc = bm25.transform(dataset.get_topics('test-2019'))
retrieved_doc[retrieved_doc.qid=='19335'].merge(test_qrels,left_on=['qid','docno'],right_on=['qid','docno'])\
.apply(lambda row:getDocuments(index,row),axis=1)

,qid,docid,docno,rank,score,query,label,documentTerms
0,19335,8412684,8412684,0,32.250450,anthropological definition of environment,3,"[{'attempt': 2}, {'understand': 2}, {'societi'..."
1,19335,7267248,7267248,1,28.072633,anthropological definition of environment,0,"[{'scientif': 1}, {'develop': 1}, {'2': 1}, {'..."
2,19335,8412687,8412687,2,27.762819,anthropological definition of environment,0,"[{'review': 2}, {'innov': 2}, {'human': 2}, {'..."
3,19335,527689,527689,3,27.700537,anthropological definition of environment,0,"[{'wai': 1}, {'matter': 2}, {'cultur': 1}, {'b..."
4,19335,3175481,3175481,4,27.698477,anthropological definition of environment,3,"[{'scienc': 1}, {'victim': 1}, {'crimin': 1}, ..."
...,...,...,...,...,...,...,...,...
134,19335,2594897,2594897,831,15.863715,anthropological definition of environment,0,"[{'scienc': 2}, {'see': 1}, {'organ': 1}, {'ef..."
135,19335,3045567,3045567,930,15.506981,anthropological definition of environment,1,"[{'scienc': 2}, {'origin': 1}, {'1': 1}, {'rel..."
136,19335,772234,772234,945,15.376898,anthropological definition of environment,0,"[{'live': 2}, {'see': 1}, {'develop': 1}, {'we..."
137,19335,6894788,6894788,985,15.361916,anthropological definition of environment,0,"[{'understand': 1}, {'caus': 1}, {'present': 2..."


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
retrieved_doc.to_csv("top1000.csv",sep="\t",header=False,index=False,columns=["qid","docid","rank"])
!cp -av 'top1000.csv' '/content/mydrive/MyDrive/Colab/Dissertation'

'top1000.csv' -> '/content/mydrive/MyDrive/Colab/Dissertation/top1000.csv'


### Query Expansion with RM3

In [ ]:
##Query Expansion with RM3
rm3_pipe = bm25 >> pt.rewrite.RM3(index,fb_terms=5,fb_docs=1000) >> bm25
retrieved_doc = rm3_pipe.transform(dataset.get_topics('test-2019'))
retrieved_doc[retrieved_doc.qid=='19335'].merge(test_qrels,left_on=['qid','docno'],right_on=['qid','docno'])\
.apply(lambda row:getDocuments(index,row),axis=1)

,qid,docid,docno,rank,score,query_0,query,label,documentTerms
0,19335,8412684,8412684,0,31.638182,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,3,"[{'attempt': 2}, {'understand': 2}, {'societi'..."
1,19335,7267248,7267248,1,29.041464,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'scientif': 1}, {'develop': 1}, {'2': 1}, {'..."
2,19335,6999135,6999135,2,28.928667,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'especi': 1}, {'cultur': 3}, {'medic': 1}, {..."
3,19335,527695,527695,3,28.913616,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'origin': 1}, {'who': 1}, {'understand': 1},..."
4,19335,527689,527689,4,28.788292,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'wai': 1}, {'matter': 2}, {'cultur': 1}, {'b..."
...,...,...,...,...,...,...,...,...,...
123,19335,2604487,2604487,884,15.122533,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'scienc': 1}, {'includ': 1}, {'approach': 2}..."
124,19335,1720388,1720388,899,15.120408,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'offic': 1}, {'1': 1}, {'affect': 1}, {'crea..."
125,19335,3212083,3212083,909,15.040660,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'scientif': 1}, {'commun': 1}, {'world': 1},..."
126,19335,5078863,5078863,936,14.923142,anthropological definition of environment,applypipeline:off human^0.051368006 environ^0....,0,"[{'resourc': 1}, {'river': 1}, {'ocean': 1}, {..."


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
retrieveddf_2 = rm3_pipe.search("human^0.042947367 attempt^0.021894734 ecolog^0.054736838 environ^0.832842118 definit^0.800000018 journal^0.021894734 understand^0.021894734 anthropolog^0.316210538 societi^0.043789469 rel^0.021894734 influenc^0.021894734")
retrieveddf_2[retrieveddf_2.qid=='1'].merge(test_qrels[test_qrels.qid=='19335'],left_on=['docno'],right_on=['docno'])\
.apply(lambda row:getDocuments(index,row),axis=1)

,qid_x,docid,docno,rank,score,query_0,query,qid_y,label,documentTerms
0,1,8412684,8412684,0,24.947486,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,3,"[{'attempt': 2}, {'understand': 2}, {'societi'..."
1,1,5078867,5078867,2,23.277332,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'medic': 1}, {'learn': 1}, {'environ': 6}, {..."
2,1,8635981,8635981,3,20.876276,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'chang': 1}, {'process': 1}, {'theori': 2}, ..."
3,1,429849,429849,4,19.745005,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'live': 2}, {'scienc': 1}, {'2': 1}, {'group..."
4,1,8412681,8412681,5,19.682697,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,2,"[{'first': 1}, {'unit': 1}, {'relationship': 1..."
...,...,...,...,...,...,...,...,...,...,...
64,1,7665056,7665056,739,13.254007,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'2': 1}, {'interact': 1}, {'desert': 1}, {'r..."
65,1,1160871,1160871,795,13.216095,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'impact': 4}, {'resourc': 1}, {'design': 1},..."
66,1,1160863,1160863,841,13.069023,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'impact': 4}, {'2': 1}, {'resourc': 1}, {'de..."
67,1,1274615,1274615,910,12.976208,human^0.042947367 attempt^0.021894734 ecolog^0...,applypipeline:off dictionari^0.030090787 attem...,19335,0,"[{'can': 1}, {'new': 1}, {'site': 1}, {'produc..."


##ColBERT Implementation

### Using ColBERT reRank result 

In [125]:
colbert_res_debasis = pd.read_csv("/content/mydrive/MyDrive/Colab/Dissertation/reranked.res.trec-debasis",sep=" ",header=None,\
            names=["qid","number","docid","rank","scpre","msg"],index_col=False)
colbert_res = pd.read_csv("/content/mydrive/MyDrive/Colab/Dissertation/colbert-ranking.tsv",sep="\t",header=None,\
            names=["qid","docid","rank"],index_col=False)

test_topics_19["qid"]=test_topics_19.qid.astype("int64")

corpus = pt.get_dataset("msmarco_passage").get_corpus()
dfcolbertcor = pd.read_csv(corpus[0],sep="\t",header=None,names=["docno", "doctext"])

colBERT_reranked_data = colbert_res.merge(test_topics_19,on="qid").apply(lambda row:getDocumentsCorpus(dfcolbertcor,row),axis=1)
colBERT_reranked_data.to_csv("ColBERT_reranked_data.csv",header=False,index=False,columns=["qid","docid","rank","query","doctext"])
!cp -av 'ColBERT_reranked_data.csv' '/content/mydrive/MyDrive/Colab/Dissertation'

collection.tar.gz:   0%|          | 0.00/987M [00:00<?, ?iB/s]

In [ ]:
## An end-to-end ColBERT dense retrieval index can be created as follows:
!rm -rf /content/colbert_indices

from pyterrier_colbert.indexing import ColBERTIndexer
from pyterrier_colbert.ranking import ColBERTFactory
import pandas as pd
import gc


checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"

corpus = pt.get_dataset("msmarco_passage").get_corpus()
dfcolbertcor = pd.read_csv(corpus[0],sep="\t",header=None,names=["docno", "text"])
test_qrels = test_qrels.astype({"docno": int}, errors='raise')
df1 = dfcolbertcor[dfcolbertcor.docno.isin(test_qrels.query("qid=='19335'").docno)]
df2 = dfcolbertcor[~dfcolbertcor.docno.isin(test_qrels.query("qid=='19335'").docno)][:90000]
dfcolbertcor = pd.concat([df2,df1])

## free memory
del df1,df2
gc.collect()

##If indexing is done offline
online_index=True
if online_index:

  indexer = ColBERTIndexer(checkpoint,"/content/colbert_indices","colbert_passage_idx",chunksize=3,ids=True)
  indexer.index(dfcolbertcor.to_dict(orient="records"))
  pyterrier_colbert_factory = indexer.ranking_factory()
  colbert_e2e = pyterrier_colbert_factory.end_to_end()

else:
  ## An end-to-end ColBERT dense retrieval pipeline can be formulated as follows:  

  pytcolbert = ColBERTFactory(checkpoint, "/content/mydrive/MyDrive/Colab/Dissertation/colbert_indices", "colbert_passage_idx")
  colbert_e2e = pytcolbert.end_to_end()

In [ ]:
!ls -ltrh colbert_indices/colbert_passage_idx

total 1.9G
-rw-r--r-- 1 root root 1.3G Jun 29 08:24 0.pt
-rw-r--r-- 1 root root  40M Jun 29 08:24 0.tokenids
-rw-r--r-- 1 root root  63M Jun 29 08:24 0.sample
-rw-r--r-- 1 root root 216K Jun 29 08:24 doclens.0.json
-rw-r--r-- 1 root root 368M Jun 29 08:28 1.pt
-rw-r--r-- 1 root root  12M Jun 29 08:28 1.tokenids
-rw-r--r-- 1 root root  19M Jun 29 08:28 1.sample
-rw-r--r-- 1 root root  63K Jun 29 08:28 doclens.1.json
-rw-r--r-- 1 root root 173K Jun 29 08:28 docnos.pkl.gz
-rw-r--r-- 1 root root 153M Jun 29 08:31 ivfpq.100.faiss


In [ ]:
## Copy indexed folder to drive for backup
!tar -zcvf colbert_indices/colbert_passage_idx.tar.gz colbert_indices/colbert_passage_idx
!cp -av 'colbert_indices/colbert_passage_idx.tar.gz' '/content/mydrive/MyDrive/Colab/Dissertation'

!tar -xzvf "/content/mydrive/MyDrive/Colab/Dissertation/colbert_passage_idx.tar.gz" -C "/content/mydrive/MyDrive/Colab/Dissertation/"

!ls -ltr /content/mydrive/MyDrive/Colab/Dissertation/colbert_indices/colbert_passage_idx/

colbert_indices/colbert_passage_idx/
colbert_indices/colbert_passage_idx/doclens.0.json
colbert_indices/colbert_passage_idx/doclens.1.json
colbert_indices/colbert_passage_idx/ivfpq.100.faiss
colbert_indices/colbert_passage_idx/1.pt
colbert_indices/colbert_passage_idx/0.tokenids
colbert_indices/colbert_passage_idx/1.sample
colbert_indices/colbert_passage_idx/docnos.pkl.gz
colbert_indices/colbert_passage_idx/0.sample
colbert_indices/colbert_passage_idx/0.pt
colbert_indices/colbert_passage_idx/1.tokenids
'colbert_indices/colbert_passage_idx.tar.gz' -> '/content/mydrive/MyDrive/Colab/Dissertation/colbert_passage_idx.tar.gz'
colbert_indices/colbert_passage_idx/
colbert_indices/colbert_passage_idx/doclens.0.json
colbert_indices/colbert_passage_idx/doclens.1.json
colbert_indices/colbert_passage_idx/ivfpq.100.faiss
colbert_indices/colbert_passage_idx/1.pt
colbert_indices/colbert_passage_idx/0.tokenids
colbert_indices/colbert_passage_idx/1.sample
colbert_indices/colbert_passage_idx/docnos.pkl.g

In [ ]:
retrieved_doc = colbert_e2e.transform(dataset.get_topics('test-2019'))

# retrieved_doc = colbert_e2e.search('anthropological definition of environment')

test_qrels = test_qrels.astype({"docno": int}, errors='raise')

retrieved_doc[retrieved_doc.qid=="19335"].merge(test_qrels,left_on=['qid','docno'],right_on=['qid','docno'])\
.apply(lambda row:getDocument(dfcolbertcor,row),axis=1).sort_values(by=['rank'])

###Query Expansion with RM3 for ColBERT

## Run Experiment

In [ ]:
def retExperiment(retr_systems,topics,qrels,metrics=['map'],dataframe=True,baseline=None,names=None,perquery=False,verbose=True):
  return pt.Experiment(
      retr_systems=retr_systems,
      topics = topics,
      qrels = qrels,
      eval_metrics=metrics,
      round=4,
      dataframe=dataframe,
      baseline=baseline,
      names = names,
      perquery = perquery,
      verbose=verbose
)
retExperiment(retr_systems = [bm25,rm3_pipe],topics=test_topics,qrels=test_qrels,baseline=0,names=["bm25","rm3"])

## Helper functions

### Functions to get posting list and raw documents

In [9]:
## Get document posting list
def getDocuments(index,row):
  docid=row["docid"]
  terms = []
  pointer = index.getDocumentIndex().getDocumentEntry(docid)
  for p in index.getDirectIndex().getPostings(pointer):
    termid = p.getId()
    term = index.getLexicon()[termid].getKey()
    # print(f"\t{term}::{p.getFrequency()}")
    terms.append({term:p.getFrequency()})
  row['documentTerms']=terms
  return row

## Fetch raw documents
def getDocumentsCorpus(corpus,row):
  row['doctext']=corpus[corpus.docno==row.docid].doctext.values[0]
  return row

### Clean up text and get parsed document text

In [11]:
## Filter records from subgroup top-k
import spacy,string,re
nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    #lower case
    processed_text = text.lower()
    # remove numbers
    processed_text = re.sub(r'\d+', '', processed_text)
    # remove punctuations and convert characters to lower case
    processed_text = processed_text.translate(str.maketrans('', '', string.punctuation)) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    processed_text = re.sub('\s+', ' ', processed_text).strip()
    return processed_text

def filterTopKRankRecord(subgroup,topK=10):
  subgroup = subgroup[subgroup['rank']<=topK]
  subgroup['doctext_parsed'] = subgroup['doctext']\
  .apply(lambda text: " ".join(token.lemma_ for token in nlp(clean_text(text)) \
                               if (not token.is_stop and not token.is_punct and token.text!=None)))
  return subgroup

### generate Corpus with IDF score

In [19]:
## function to calculate corpus and respective IDF for every query

import math

def generateCorpusIDF(documentsdf):
  totaldocument = documentsdf.shape[0]
  corpus = Counter()
  for doc in documentsdf.iterrows():
    for term in set(doc[1].doctext_parsed.split(" ")):
      corpus[term] +=1 
  idf = {}
  for term,termdoccount in corpus.items():
    inversedocfreq = math.log(totaldocument/termdoccount+1)
    idf[term] = inversedocfreq
  return idf



## ColBERT and BM25 comparison

### Check the similarity between BM25 with Query Expansion and ColBERT

In [14]:
colbert_reranked_docs = pd.read_csv('/content/mydrive/MyDrive/Colab/Dissertation/ColBERT_reranked_data.csv',
                                    names=["qid","docid","rank","query","doctext"])
colbert_docs_50 = colbert_reranked_docs.groupby("qid",as_index=False).apply(lambda subgroup:filterTopKRankRecord(subgroup,topK=50))

In [20]:
##subset term selection using tfidf

from collections import Counter

count=0
current_qid=""
count_dict={}

queryCorpus = {}
for name,group in colbert_docs_50.groupby("qid",as_index=False):
  queryCorpus[name] = generateCorpusIDF(group)


#Calculating tfidf
querydoc = {}
for key in queryCorpus.keys():
  docs={}
  for doc in colbert_docs_50[colbert_docs_50.qid == key].iterrows():
    tfidf={}
    docterms = Counter(doc[1].doctext_parsed.split(" "))
    docLength = len(doc[1].doctext_parsed.split(" "))
    for term in queryCorpus[key]:
      if not term in docterms:
        tfidf[term] = 0.0
      else:
        tfidf[term] = float(docterms[term]/docLength)*queryCorpus[key][term] #tfidf
    docs[doc[1].docid]=tfidf
  querydoc[key]=docs

In [156]:
## code to fetch terms from corpus and check the jaccard simiarity between the BM25 with expanded query and ColBERT
import os
from tqdm import tqdm


def calTermWeights(queryVocabularies,base,colbert_docs_50,similarity_type):
  jaccsimilarity = []
  for qid in tqdm(queryVocabularies.keys()):
    queryVocab = queryVocabularies[qid]
    query = colbert_docs_50[colbert_docs_50.qid==qid]['query'].unique()[0]
    for word in queryVocab:
      expanded_query = " ".join([query,word])
      retrieved_doc = bm25.search(expanded_query)
      retrieved_doc.qid=qid
      jscore = similarity(retrieved_doc,colbert_docs_50,type=similarity_type)
      diff = checkIncrement(base[qid],jscore[qid])
      jaccsimilarity.append((qid,word,expanded_query,jscore,diff))
  return jaccsimilarity

#function to calculate similarity measure
def similarity(bm25,colbert,type="jaccard"):
  score = {}
  if type.lower() == "jaccard":
    for qid in bm25.qid.unique():
      bm25querysubset = set(bm25[bm25.qid==qid]['docid'])
      colbertquerysubset = set(colbert[colbert.qid==int(qid)]['docid'])
      jaccardscore = float(len(bm25querysubset.intersection(colbertquerysubset))) / len(bm25querysubset.union(colbertquerysubset))
      score[int(qid)]=jaccardscore
  else:
    print(f"{type} similarity yet to be implemented.")
  return score

#function to calculate improvement
def checkIncrement(baseJaccardScore,currentJaccardScore):
  #check relative increment in the jaccard similarity score for each term
  return float((currentJaccardScore-baseJaccardScore)/baseJaccardScore)


## get baseline performance
bm25 = pt.BatchRetrieve(index,num_results=50, wmodel="BM25")
retrieved_doc = bm25.transform(test_topics)
base = similarity(retrieved_doc,colbert_docs_50)

#initiate the process
result = calTermWeights(queryCorpus,base,colbert_docs_50,'jaccard')
df = pd.DataFrame(result, columns =['qid','word','expanded_query','jscore','improvement'])
df.to_csv("ColBERT_BM25_comparison.csv",header=['qid','word','expanded_query','jscore','improvement'],index=False)
!cp -av 'ColBERT_BM25_comparison.csv' '/content/mydrive/MyDrive/Colab/Dissertation'

100%|██████████| 43/43 [23:47<00:00, 33.19s/it]


'ColBERT_BM25_comparison.csv' -> '/content/mydrive/MyDrive/Colab/Dissertation/ColBERT_BM25_comparison.csv'
